In [1]:
from binance import Client
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
with open('access.txt') as f:
    acc = f.readlines()
api = acc[0].strip()
key = acc[1].strip()

In [3]:
client = Client(api,key)

In [4]:
engine = sqlalchemy.create_engine('sqlite:///DogeTrade.db')

In [14]:
class DogeTrade:
    def __init__(self, lookback, qty, position=False):
        self.lookback = lookback
        self.qty = qty
        self.position = position
        self.buy_orders = []
        self.sell_orders = []
        self.engine = sqlalchemy.create_engine('sqlite:///DogeTrade.db')

    def buy_order(self):
        while True:
            data = pd.read_sql('doge_test_1', self.engine)
            data_lookback = data.iloc[-self.lookback:]
            change = (data_lookback.Price.pct_change() +1).cumprod() - 1
            if change[change.last_valid_index()] < -0.003:
                order = client.create_order(symbol='DOGEUSDT',side='BUY', type='MARKET', quantity=self.qty)
                print('BUY order executed!')
                print(order)
                self.position = True
                self.buy_orders.append(order)
                break
    
    def sell_order(self):
        time_buy = pd.to_datetime(self.buy_orders[-1]['transactTime'],unit='ms') + pd.Timedelta('03:00:00')
        while True:
            data = pd.read_sql('doge_test_1', self.engine)
            since_buy = data.loc[data.Time > time_buy]
            if len(since_buy) > 1:
                change = (since_buy.Price.pct_change() +1).cumprod() -1
                if change[change.last_valid_index()] > 0.005:
                    order = client.create_order(symbol='DOGEUSDT',side='SELL', type='MARKET', quantity=self.qty)
                    print('SELL order executed')
                    print(order)
                    self.position = False
                    self.sell_orders.append(order)
                    break
                  
    def trade(self):
        while len(self.sell_orders) < 2:
            if not self.position:
                self.buy_order()
            if self.position:
                self.sell_order()

In [15]:
doge_trade = DogeTrade(500, 50)
doge_trade.trade()

BUY order executed!
{'symbol': 'DOGEUSDT', 'orderId': 1453012834, 'orderListId': -1, 'clientOrderId': 'rfl5p4nRAtZEAeutjmmbjJ', 'transactTime': 1626177400671, 'price': '0.00000000', 'origQty': '50.00000000', 'executedQty': '50.00000000', 'cummulativeQuoteQty': '10.13800000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '0.20276000', 'qty': '50.00000000', 'commission': '0.05000000', 'commissionAsset': 'DOGE', 'tradeId': 287811917}]}


KeyboardInterrupt: 

In [174]:
b = [float(i['cummulativeQuoteQty']) for i in doge_trade.buy_orders]
s = [float(i['cummulativeQuoteQty']) for i in doge_trade.sell_orders]